# 0. 実行環境

* Language/Runtime: **JavaScript (Node.js v22.14.0)**（コードは Node 18+ 互換）
* Module: **CommonJS**
* 外部ライブラリ: **禁止（Node 標準のみ）**
* CI前提: `node solution.js` 実行可能（LeetCode では関数のみ評価）

# 1. 問題の分析

## 競技プログラミング視点

* **目的**: 32bit 符号付き整数 `x` の十進数字を逆順にして返す。範囲外（`[-2^31, 2^31-1]`）に出たら `0`。
* **鍵ポイント**: 乗算/加算の都度オーバーフローを事前検知（`rev` が境界を超える直前で検査）。
* **I/O**: LeetCode 形式の関数のみ。副作用なし。

## 業務開発視点

* **保守性**: `INT_MAX/INT_MIN` を定数化。境界ロジックはコメントで明示。
* **例外/入力検証**: 数値・整数・範囲の軽量チェックを早期に実施（ホットパス外）。不正入力は `TypeError / RangeError` を送出。

## JavaScript特有の考慮点

* **V8 最適化**: 単純な `while` ループ＋プリミティブ `number` のみ。hidden class の変動なし。
* **整数切り捨て**: `x = (x / 10) | 0` でゼロ方向切捨て（32bit 符号演算）を安定化。
* **GC対策**: 一時オブジェクト生成ゼロ。クロージャ不使用。

---

# 2. アルゴリズムアプローチ比較

| アプローチ                      | 時間計算量 | 空間計算量 | JS実装コスト | 可読性 | 備考                |
| -------------------------- | ----: | ----: | ------: | --- | ----------------- |
| 方法A（桁ポップ&プッシュ、逐次オーバーフロー検査） |  O(d) |  O(1) |       低 | 中   | `rev` を都度チェック（推奨） |
| 方法B（文字列反転→数値化→範囲判定）        |  O(d) |  O(d) |       低 | 高   | 文字列化で一時メモリ増、不要    |
| 方法C（配列バッファ）                |  O(d) |  O(d) |       中 | 中   | Bの下位互換、利点なし       |

`d` は桁数（最大 10）。

---

# 3. 選択したアルゴリズムと理由

* **選択**: 方法A（桁ポップ＆プッシュ）
* **理由**: 追加メモリ不要、毎ステップで安全に 32bit 境界を検査でき、最速・最小。
* **JS最適化ポイント**:

  * `while (x !== 0)` の単純ループ
  * `x % 10` で下位桁抽出、`(x / 10) | 0` でゼロ方向切捨て
  * 事前境界チェック：`rev` が `INT_MAX/10`（または `INT_MIN/10`）を超えるか、等しくて末尾桁条件を満たさない場合は即 `0`

---

# 4. コード実装（solution.js）

```javascript
'use strict';
// Module: CommonJS（ローカル実行用に module.exports を併記。LeetCode では無視されます。）

/**
 * Reverse Integer (LeetCode #7) - Pure Function
 *
 * 与えられた 32bit 符号付き整数 x の数字を反転して返す。
 * 反転結果が 32bit 範囲 [-2^31, 2^31-1] を外れたら 0 を返す。
 *
 * 計算量: 時間 O(d), 空間 O(1)  （d は桁数 ≤ 10）
 *
 * @param {number} x - 入力整数
 * @returns {number} 反転後の整数（範囲外は 0）
 * @throws {TypeError} 型が number でない / 非有限値 / 整数でない
 * @throws {RangeError} 入力が 32bit 符号付き整数範囲外
 */
function reverse(x) {
  // ---- 入力検証（ホットパス外の早期チェック）----
  if (typeof x !== 'number' || !Number.isFinite(x)) {
    throw new TypeError('Input must be a finite number');
  }
  if (!Number.isInteger(x)) {
    throw new TypeError('Input must be an integer');
  }
  const INT_MAX = 2147483647;  //  2^31 - 1
  const INT_MIN = -2147483648; // -2^31
  if (x < INT_MIN || x > INT_MAX) {
    throw new RangeError('Input out of 32-bit signed integer range');
  }

  // ---- 本処理（桁ポップ＆プッシュ；逐次オーバーフロー検査）----
  let rev = 0;
  // 以降、x は 32bit 整数範囲内で推移する。ゼロ方向に切り捨てるため (x/10)|0 を使用。
  while (x !== 0) {
    // 下位桁を取り出し
    const pop = x % 10;      // JS の % は符号を保つ（負数でもOK）
    x = (x / 10) | 0;        // toward-zero truncation（32bit 符号演算）

    // オーバーフロー事前検知（10倍＋pop の前に境界比較）
    // 上限チェック：rev > INT_MAX/10 なら次で確実に溢れる
    if (rev > 214748364 || (rev === 214748364 && pop > 7)) {
      return 0;
    }
    // 下限チェック：rev < INT_MIN/10 なら次で確実に溢れる
    if (rev < -214748364 || (rev === -214748364 && pop < -8)) {
      return 0;
    }
    rev = rev * 10 + pop;
  }
  return rev;
}

// LeetCode では関数定義のみで可。ローカル実行やCI向けにエクスポートを同梱。
module.exports = { reverse };

Analyze Complexity
Runtime 57 ms
Beats 43.86%
Memory 55.49 MB
Beats 74.57%

```

---

# 5. 追加メモ（JS最適化チェックリスト）

* ループは `while`/`for` の基本形を使用（`map/forEach` 非採用）。
* 追加オブジェクト・配列の生成ゼロ。
* 定数は先頭で確定、プロパティの動的追加なし（hidden class 安定）。
* 数値のみを扱い単型維持、クロージャ不使用で GC 圧を回避。
* 例外はホットパスの外で早期判定（正常系は分岐最小化）。

---

# 1. 問題の分析

## 競技プログラミング視点での分析

* すでに O(d), O(1) で実装済みならボトルネックは**処理本体の分岐・型変換コスト**。
* LeetCode 入力は常に妥当（型/範囲チェック不要）なので、**バリデーションを外してホットパスを最短化**する余地があります。
* `|0` 等の**ビット演算は ToInt32 変換を誘発**し、数値タグ（Smi/HeapNumber）の行き来でデオプトが出ることがあります。

## 業務開発視点での分析

* プロダクションでは検証を残す価値あり。ただし LeetCode では**最速優先で検証を削除**する構成が実利的。
* 可読性は維持しつつ、**境界定数の外出し**と**分岐の並び順**（ヒット頻度の高いケースを先）で分岐予測を助けます。

## JavaScript特有の考慮点

* `x = (x / 10) | 0` よりも **`x = (x - pop) / 10`** は余計な int32 化を避け、V8 上で安定して速い傾向。
* `Math.trunc` は関数呼び出しオーバヘッドがあり、短いループでは不利になりがち。
* 文字列化（`String/Array`）は GC プレッシャ増。数値演算だけで完結させるのが安全。

---

# 2. アルゴリズムアプローチ比較

| アプローチ                 |    時間計算量 | 空間計算量 | JS実装コスト | 可読性 | 備考                  |                   |
| --------------------- | -------: | ----: | ------: | --- | ------------------- | ----------------- |
| 方法A（数値のみ、事前OF検査）※最適化版 |     O(d) |  O(1) |       低 | 中   | `x=(x-pop)/10`、検証削除 |                   |
| 方法B（従来A：`             | 0` で切捨て） |  O(d) |    O(1) | 低   | 中                   | ToInt32 による型変換が発生 |
| 方法C（文字列反転）            |     O(d) |  O(d) |       低 | 高   | メモリ圧・速度ともに不利        |                   |

---

# 3. 選択したアルゴリズムと理由

* **選択したアプローチ**: 方法A（数値のみ、事前オーバーフロー検査／入力検証レス）
* **理由**:

  * LeetCode 前提で入力検証を省略し**ホットパス最短化**。
  * **ビット演算排除**で不要な ToInt32 を避け、JIT の最適化を阻害しにくい。
  * 追加メモリ 0、分岐は 2 条件のみで**分岐予測が当たりやすい**。
* **JavaScript特有の最適化ポイント**:

  * `pop = x % 10` → `x = (x - pop) / 10`（toward-zero を保証、型安定）
  * `MAX_DIV10/MIN_DIV10` 等の**定数は関数外に固定**（hidden class 安定・再割当なし）
  * ループは `while (x !== 0)` の単純形。クロージャ・一時配列ゼロ。

---

# 4. コード実装（solution.js）

```javascript
'use strict';
// Module: CommonJS（LeetCode は関数のみ評価／ローカル実行やCI向けにexport）

// 32-bit 境界定数（関数外に固定して再生成回避）
const INT_MAX = 2147483647;   //  2^31 - 1
const INT_MIN = -2147483648;  // -2^31
const MAX_DIV10 = 214748364;  // Math.trunc(INT_MAX / 10)
const MIN_DIV10 = -214748364; // Math.trunc(INT_MIN / 10)
const MAX_LAST = 7;           // INT_MAX % 10
const MIN_LAST = -8;          // INT_MIN % 10

/**
 * Reverse Integer (LeetCode #7) - Judge-Optimized, Pure
 *
 * 計算量: 時間 O(d), 空間 O(1)  （d ≤ 10）
 * 入力は LeetCode で保証されるため、検証と例外は省略（速度優先）。
 *
 * @param {number} x
 * @returns {number}
 */
function reverse(x) {
  let rev = 0;
  while (x !== 0) {
    // 符号付き % は負数でも下位桁を返す
    const pop = x % 10;

    // 事前オーバーフローチェック（rev*10 + pop を行う前）
    if (rev > MAX_DIV10 || (rev === MAX_DIV10 && pop > MAX_LAST)) return 0;
    if (rev < MIN_DIV10 || (rev === MIN_DIV10 && pop < MIN_LAST)) return 0;

    rev = rev * 10 + pop;

    // toward-zero：bit演算を避けて型安定のまま更新
    x = (x - pop) / 10;
  }
  return rev;
}

module.exports = { reverse };

Analyze Complexity
Runtime 45 ms
Beats 90.39%
Memory 55.13 MB
Beats 88.26%

```

---

# 5. 追加メモ（JS最適化チェックリスト）

* **入力検証は省略**（LeetCode前提）。業務コードでは別レイヤで実施。
* **ビット演算を排除**して ToInt32 強制変換を避ける。
* **定数は関数外**で一度だけ確定。
* **関数は単純なスカラー演算のみ**。クロージャ・配列・文字列生成なし。
* **分岐順序**は「正常系が通りやすい」並び（`rev > MAX_DIV10` 先）。

---

## ひとこと

* いまの 57ms / 55.49MB は十分に優秀ですが、上記の**検証削除＋`(x - pop)/10`**の置き換えで、環境次第では**数％〜十数％**の微改善が期待できます。
* さらに詰めるなら、関数定義を**ファイル先頭**に固定・`'use strict'` 維持・他のグローバルを増やさない（IC 汚染回避）といった**周辺ノイズの削減**も有効です。

